# Creating the shots dataset.

#### Since the model has to span over multiple games, this notebook will be used to collect the shot-events from each match file, and aggregate it in a single dataset.



### Step 1 - Importing Necessary Modules

In [125]:
import os
import csv
import pandas as pd
import win32com.client
import numpy as np

### Step 2 - Converting to CSV

In [28]:
# As the default setting of the collection software stores the event data as '.xls' extensions, this cell will
# be used to convert each file into a '.csv' file for easier use.

xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
filePath = "0path"

for root,dirs,files in os.walk(filePath):
    for file in files:
        if file.endswith(".xls"):
            x = os.path.join(root,file)
            xl.Workbooks.Open(os.path.join(root,file))
            xl.ActiveWorkbook.SaveAs(Filename=x.replace('.xls', '.csv'),
                                     FileFormat=win32com.client.constants.xlCSVMSDOS,
                                     CreateBackup=True)
            xl.ActiveWorkbook.Close(SaveChanges=True)

### Step 3 - Extracting the Shots.

In [29]:
# the code in this cell creates a "masterfile", and appends any rows with pass or shot events to it.

filesList = []
writer = open("xG-shots-master.csv", 'w')
writer = csv.writer(writer, delimiter='\t')
for root,dirs,files in os.walk(filePath):
    for file in files:
        if file.endswith(".csv"):
            filesList.append(file)
            f = open(os.path.join(root,file), "r")
            reader = csv.reader(f, delimiter='\t')

            for column in reader:
                # print(column[0])
                column_list = column[0].split(',')
                # print(column_list)
                if ("Pass" in column_list[1] or "Goal" in column_list[1] or "Shot" in column_list[1]) and ("Kick" not in column_list[1]):
                    writer.writerow(column)

In [39]:
df = pd.read_csv("xG-shots-master.csv")
df.head()

,1,R Grounded Pass,53:14:13,53:16:13,53:17:13,Unnamed: 5,TG,d7,TG.1,a9,...,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155
0,2,R Grounded Pass,53:16:19,53:18:19,53:19:19,NaN,G6,a10,G3,e12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,R Grounded Pass,53:19:03,53:21:03,53:22:03,NaN,G3,e12,G2,g6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,R Grounded Pass,53:26:15,53:28:15,53:29:15,NaN,B5,c3,B2,d2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,R Grounded Pass,53:30:02,53:32:02,53:33:02,NaN,B2,e2,TB,a5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,R Grounded Pass,53:32:04,53:34:04,53:35:04,NaN,TB,a5,B5,c2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 3.1 - Cleaning the Dataframe

In [98]:
# There are some issues with this dataframe:
# 1) there is no header row [The headers can be taken from one of the collected match files]
# 2) the last ~150 columns are useless
# 3) column headers need to be changed to make them more usable

# Let's deal with that. 

df2 = pd.read_csv("C:\\Users\\91953\\Desktop\\InTheGame\\Match Files\\020120\\2200 - 2300\\22020120 - Part 4.csv")
df.columns = df2.columns
df3 = df.iloc[:,:-144]
df3.columns = df3.columns.str.replace(' ', '_')
df3.head()

,N,Category,Start,Click,End,Descriptors,Des_1,Des_2,Des_3,Des_4,Des_5,Des_6
0,2,R Grounded Pass,53:16:19,53:18:19,53:19:19,NaN,G6,a10,G3,e12,NaN,NaN
1,3,R Grounded Pass,53:19:03,53:21:03,53:22:03,NaN,G3,e12,G2,g6,NaN,NaN
2,4,R Grounded Pass,53:26:15,53:28:15,53:29:15,NaN,B5,c3,B2,d2,NaN,NaN
3,5,R Grounded Pass,53:30:02,53:32:02,53:33:02,NaN,B2,e2,TB,a5,NaN,NaN
4,6,R Grounded Pass,53:32:04,53:34:04,53:35:04,NaN,TB,a5,B5,c2,NaN,NaN


### Step 4 - Data Manipulation

#### Turning the above dataframe to the final shots dataset that we will use.

In [99]:
# Things that need to be done:

# 1) New columns: isShot, shotDist, shotAng, isOnTarget, goalLoc, isGoal, isHeader, isThroughball, isBigChance, isCounter,isTapIn.
# 2) remove the unnecessary rows (passes) and columns.

df2.columns = df2.columns.str.replace(' ', '_')
df4 = df2.iloc[:,:-144]
df4.head()

,N,Category,Start,Click,End,Descriptors,Des_1,Des_2,Des_3,Des_4,Des_5,Des_6
0,1,Goal Kick,53:11:14,53:13:14,53:14:14,NaN,G3,e12,TG,d7,NaN,NaN
1,1,R Grounded Pass,53:14:13,53:16:13,53:17:13,NaN,TG,d7,TG,a9,NaN,NaN
2,2,R Grounded Pass,53:16:19,53:18:19,53:19:19,NaN,G6,a10,G3,e12,NaN,NaN
3,3,R Grounded Pass,53:19:03,53:21:03,53:22:03,NaN,G3,e12,G2,g6,NaN,NaN
4,1,Duel - Won,53:19:24,53:21:24,53:22:24,NaN,G2,g6,B3,NaN,NaN,NaN


### Step 4.1 - Calculating shot distance and shot angle using Des_2.

In [108]:
# How:
# 1) splitting the des_2 column, into separate alphabet and numeral columns.
# 2) changing the alphabets to respective numbers for the X co-ords.
# 3) finding out which goal (e0 or e13) the ball was shot at:
    # shots are generally shot to the goal closer to them, unless they are marked with a long range tag.
# 4) calculating the shot distance using pythagoras theorem.
# 5) calculating the angle using numpy arcsin.

ShotDict = {'a':'1','b':'2','c':'3','d':'4','e':'5','f':'6','g':'7','h':'8','i':'9'}
ShotDict1 = {'12':'1','11':'2','10':'3','9':'4','8':'5','7':'6','6':'7','5':'8','4':'9','3':'10','2':'11','1':'12'}
df3['shotXPrelim'] = df3.Des_2.str[0]
df3['shotYPrelim'] = df3.Des_2.str[1:]
df3['shotX'] = ''
df3['shotY'] = ''

for val in range(len(df3)):
    for key in df3.shotXPrelim.loc[val]:
        try:
            df3['shotX'].loc[val] = ShotDict[key]
        except:
            df3['shotX'].loc[val] = '0'

for val in range(len(df3)):
    for key in df3.shotYPrelim.loc[val]:
        if df3.Des_4.loc[val] == 'Long Range' or df3.Des_5.loc[val] == 'Long Range':
            try:
                df3['shotY'].loc[val] = ShotDict1[key]
            except:
                df3['shotY'].loc[val] = '0'
        else:
            try:
                df3['shotY'].loc[val] = df3['shotYPrelim'].loc[val]
            except:
                df3['shotY'].loc[val] = '0'

# changing columns to ideal dtypes
df3 = df3.astype({'shotYPrelim':'Int64','shotY':'Int64','shotX':'Int64', 'Des_4':'str', 'Des_5':'str'})

# marking the correct goalpoint
df3['goalpoint'] = df3.apply(lambda row: 0 if row.shotYPrelim <=6 else 13, axis=1)

# finding shotDist and shotAng
df3['distY'] = abs(df3['goalpoint'] - df3['shotY'])
df3['distX'] = abs(5 - df3['shotX'])
df3['shotDist'] = df3.apply(lambda row: (row.distX**2 + row.distY**2)**0.5, axis=1)
df3['shotAng'] = df3.apply(lambda row: np.degrees(np.arcsin(row.distY/row.shotDist)), axis=1)

c:\users\91953\python38-32\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,N,Category,Start,Click,End,Descriptors,Des_1,Des_2,Des_3,Des_4,Des_5,Des_6,shotXPrelim,shotYPrelim,shotX,shotY
0,2,R Grounded Pass,53:16:19,53:18:19,53:19:19,NaN,G6,a10,G3,e12,NaN,NaN,a,10,1,10
1,3,R Grounded Pass,53:19:03,53:21:03,53:22:03,NaN,G3,e12,G2,g6,NaN,NaN,e,12,5,12
2,4,R Grounded Pass,53:26:15,53:28:15,53:29:15,NaN,B5,c3,B2,d2,NaN,NaN,c,3,3,3
3,5,R Grounded Pass,53:30:02,53:32:02,53:33:02,NaN,B2,e2,TB,a5,NaN,NaN,e,2,5,2
4,6,R Grounded Pass,53:32:04,53:34:04,53:35:04,NaN,TB,a5,B5,c2,NaN,NaN,a,5,1,5


In [126]:
df3.head()

,N,Category,Start,Click,End,Descriptors,Des_1,Des_2,Des_3,Des_4,...,Des_6,shotXPrelim,shotYPrelim,shotX,shotY,goalpoint,distY,distX,shotDist,shotAng
0,2,R Grounded Pass,53:16:19,53:18:19,53:19:19,NaN,G6,a10,G3,e12,...,NaN,a,10,1,10,13,3,4,5.000000,36.869898
1,3,R Grounded Pass,53:19:03,53:21:03,53:22:03,NaN,G3,e12,G2,g6,...,NaN,e,12,5,12,13,1,0,1.000000,90.000000
2,4,R Grounded Pass,53:26:15,53:28:15,53:29:15,NaN,B5,c3,B2,d2,...,NaN,c,3,3,3,0,3,2,3.605551,56.309932
3,5,R Grounded Pass,53:30:02,53:32:02,53:33:02,NaN,B2,e2,TB,a5,...,NaN,e,2,5,2,0,2,0,2.000000,90.000000
4,6,R Grounded Pass,53:32:04,53:34:04,53:35:04,NaN,TB,a5,B5,c2,...,NaN,a,5,1,5,0,5,4,6.403124,51.340192


### Step 4.2 - Creating the other columns.

In [140]:
# Creating the other columns
# New columns: isShot, isOnTarget, goalLoc, isGoal, isHeader, isThroughball, isBigChance, isCounter,isTapIn.

df3['isShot'] = df3.apply(lambda row: 1 if 'Shot' in row.Category or 'Goal' in row.Category else 0, axis=1)
df3['isOnTarget'] = df3.apply(lambda row: 1 if 'On' in row.Category or 'Goal' in row.Category else 0, axis=1)
df3['isGoal'] = df3.apply(lambda row: 1 if 'Goal' in row.Category else 0, axis=1)
df3['isHeader'] = df3.apply(lambda row: 1 if 'H ' in row.Category else 0, axis=1)
df3['isBigChance'] = df3.apply(lambda row: 1 if 'Big' in row.Des_4 or 'Big' in row.Des_5 else 0, axis=1)
df3['isCounter'] = df3.apply(lambda row: 1 if 'Counter' in row.Des_4 or 'Counter' in row.Des_5 else 0, axis=1)
df3['isTapIn'] = df3.apply(lambda row: 1 if 'Tap' in row.Des_4 or 'Tap' in row.Des_5 else 0, axis=1)

# since pass types are collected in the previous event, we use the shift() function to obtain whether the pass was a throughball
df3['prevD5'] = df3['Des_5'].shift(1).astype(str)
df3['prevD6'] = df3['Des_6'].shift(1).astype(str)
df3['isThroughball'] = df3.apply(lambda row: 1 if 'Through' in row.prevD5 or 'Through' in row.prevD6 else 0, axis=1)
df3['goalLoc'] = df3.Des_3

# Test code to see if the changes are made
# dftest = df3[df3['Des_4'] == 'Counter Attack']
# dftest.head()

,N,Category,Start,Click,End,Descriptors,Des_1,Des_2,Des_3,Des_4,...,distX,shotDist,shotAng,isShot,isOnTarget,isGoal,isHeader,isBigChance,isCounter,isTapIn
872,1,R Goal,46:05:06,46:07:06,46:08:15,NaN,TG,c10,s8,Counter Attack,...,2,3.605551,56.309932,1,1,1,0,0,1,0
1527,4,R Goal,44:34:21,44:36:21,44:40:12,NaN,B5,c4,s11,Counter Attack,...,2,4.472136,63.434949,1,1,1,0,0,1,0
2010,2,R Goal,46:43:16,46:45:16,46:46:17,NaN,TB,f11,s21,Counter Attack,...,1,2.236068,63.434949,1,1,1,0,0,1,1


### Step 5 - Removing excess columns and rows.

In [172]:
# Removing non-shots for training.

dffinal = df3[df3['isShot']==1]
dffinal.head()

,N,Category,Start,Click,End,Descriptors,Des_1,Des_2,Des_3,Des_4,...,isOnTarget,isGoal,isHeader,isBigChance,isCounter,isTapIn,prevD5,prevD6,isThroughball,goalLoc
9,1,R Goal,53:47:04,53:49:04,53:50:04,NaN,TG,e8,s11,Long Range,...,1,1,0,0,0,0,nan,nan,0,s11
12,2,R Goal,54:11:03,54:13:03,54:14:03,NaN,B3,f11,s17,nan,...,1,1,0,0,0,0,nan,nan,0,s17
17,1,L Shot On,54:34:03,54:36:03,54:37:03,NaN,G4,g4,s21,nan,...,1,0,0,0,0,0,nan,nan,0,s21
21,1,R Shot On,55:06:24,55:08:24,55:09:24,NaN,G4,d4,s16,nan,...,1,0,0,0,0,0,nan,nan,0,s16
25,2,R Shot On,55:20:07,55:22:07,55:23:07,NaN,B2,e10,s21,nan,...,1,0,0,0,0,0,Through Ball,nan,1,s21


In [167]:
# removing the columns that won't be used for training.
dffinalexp = dffinal.drop(dffinal.columns[[0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,28,29]], axis=1)  # df.columns is zero-based pd.Index 

,Des_1,shotDist,shotAng,isShot,isOnTarget,isGoal,isHeader,isBigChance,isCounter,isTapIn,isThroughball,goalLoc
9,TG,8.000000,90.000000,1,1,1,0,0,0,0,0,s11
12,B3,2.236068,63.434949,1,1,1,0,0,0,0,0,s17
17,G4,4.472136,63.434949,1,1,0,0,0,0,0,0,s21
21,G4,4.123106,75.963757,1,1,0,0,0,0,0,0,s16
25,B2,3.000000,90.000000,1,1,0,0,0,0,0,1,s21


In [173]:
# saving the training datasets

dffinal.to_csv('xgdata.csv')
dffinalexp.to_csv('finalxgdata.csv')

In [171]:
# test
xgdata = pd.read_csv('finalxgdata.csv')
xgdata.head()

,Unnamed: 0,Des_1,shotDist,shotAng,isShot,isOnTarget,isGoal,isHeader,isBigChance,isCounter,isTapIn,isThroughball,goalLoc
0,9,TG,8.000000,90.000000,1,1,1,0,0,0,0,0,s11
1,12,B3,2.236068,63.434949,1,1,1,0,0,0,0,0,s17
2,17,G4,4.472136,63.434949,1,1,0,0,0,0,0,0,s21
3,21,G4,4.123106,75.963757,1,1,0,0,0,0,0,0,s16
4,25,B2,3.000000,90.000000,1,1,0,0,0,0,0,1,s21
